<a href="https://colab.research.google.com/github/ninja-marduk/ml_precipitation_prediction/blob/main/models/base_models_gnn_convlstm_stacking_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# V5 GNN-ConvLSTM Stacking: Meta-Learner Ensemble for Spatiotemporal Precipitation Prediction

## Novel Hybrid Architecture - Dual-Branch Stacking with Interpretable Meta-Learner

**Version:** 5.0  
**Date:** January 2026  
**Author:** Manuel Perez  
**Institution:** UPTC - Doctoral Thesis in Engineering  

---

### Architecture Overview

V5 combines the complementary strengths of V2 (ConvLSTM) and V4 (GNN-TAT) through a stacking ensemble:

| Component | Description | Innovation |
|-----------|-------------|------------|
| **Branch 1: ConvLSTM** | Euclidean spatial patterns (BASIC features) | Local spatiotemporal extraction |
| **Branch 2: GNN-TAT** | Non-Euclidean topographic relations (KCE features) | Graph-based orographic modeling |
| **Grid-Graph Fusion** | Cross-attention between representations | Novel grid↔graph alignment |
| **Meta-Learner** | Interpretable weighted fusion | Context-dependent branch weighting |

### Target Performance (from spec.md)

| Metric | V4 Baseline | V5 Target | V5 Excellent |
|--------|-------------|-----------|---------------|
| R² (H1-H6) | 0.628 | > 0.65 | > 0.70 |
| RMSE (mm) | 92.12 | < 85 | < 80 |
| Parameters | 98K | < 200K | < 180K |

### Innovation Status
**NOVEL CONTRIBUTION:** No existing Q1 publications combine GNN and ConvLSTM in stacking ensemble for precipitation prediction (verified January 2026).

## 1. Environment Setup and Imports

In [ ]:
# =============================================================================
# SECTION 1.1: ENVIRONMENT DETECTION AND SETUP
# =============================================================================

import os
import sys
from pathlib import Path

# Detect environment
IN_COLAB = 'google.colab' in sys.modules
print(f"Running in Google Colab: {IN_COLAB}")

if IN_COLAB:
    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')

    # Install required packages (match PyTorch/CUDA build)
    import torch

    torch_version = torch.__version__.split('+')[0]
    cuda_version = torch.version.cuda
    if cuda_version:
        cuda_tag = f"cu{cuda_version.replace('.', '')}"
    else:
        cuda_tag = "cpu"
    pyg_url = f"https://data.pyg.org/whl/torch-{torch_version}+{cuda_tag}.html"
    print(f"Installing PyG wheels from: {pyg_url}")
    !pip install -q torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f {pyg_url}
    !pip install -q netCDF4 xarray dask h5netcdf

    # Set base paths for Colab
    BASE_PATH = Path('/content/drive/MyDrive/ml_precipitation_prediction')
    DRIVE_DATA_FILE = BASE_PATH / 'data' / 'output' / 'complete_dataset_with_features_with_clusters_elevation_windows_imfs_with_onehot_elevation_clean.nc'
    LOCAL_DATA_FILE = Path('/content/complete_dataset_with_features_with_clusters_elevation_windows_imfs_with_onehot_elevation_clean.nc')
    OUTPUT_ROOT = BASE_PATH / 'models' / 'output' / 'V5_GNN_ConvLSTM_Stacking'

    # Copy dataset to local for faster access
    if not LOCAL_DATA_FILE.exists():
        !cp "{DRIVE_DATA_FILE}" "{LOCAL_DATA_FILE}"
        print("Dataset copied to local storage for faster access")
    DATA_FILE = LOCAL_DATA_FILE
else:
    # Local paths
    BASE_PATH = Path('d:/github.com/ninja-marduk/ml_precipitation_prediction')
    DATA_FILE = BASE_PATH / 'data' / 'output' / 'complete_dataset_with_features_with_clusters_elevation_windows_imfs_with_onehot_elevation_clean.nc'
    OUTPUT_ROOT = BASE_PATH / 'models' / 'output' / 'V5_GNN_ConvLSTM_Stacking'

# Create output directories
OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)
print(f"Output directory: {OUTPUT_ROOT}")


Running in Google Colab: False
Output directory: d:\github.com\ninja-marduk\ml_precipitation_prediction\models\output\V5_GNN_ConvLSTM_Stacking


In [2]:
# =============================================================================
# SECTION 1.2: IMPORTS
# =============================================================================

import json
import gc
import copy
import warnings
from datetime import datetime
from dataclasses import dataclass, field, asdict
from typing import Dict, List, Tuple, Optional, Any

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

# PyTorch Geometric
from torch_geometric.nn import GCNConv, GATConv, SAGEConv
from torch_geometric.data import Data, Batch

from sklearn.cluster import KMeans
from scipy import stats

warnings.filterwarnings('ignore')

# Set seeds for reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")


Using device: cuda
GPU: NVIDIA GeForce GTX 960M
GPU Memory: 4.3 GB


## 2. Configuration

In [3]:
# =============================================================================
# SECTION 2: V5 CONFIGURATION (from spec.md Section 3.3)
# =============================================================================

@dataclass
class V5Config:
    """Complete V5 GNN-ConvLSTM Stacking configuration."""

    # Data configuration
    input_window: int = 60      # Input months
    horizon: int = 12           # Prediction months
    train_val_split: float = 0.8

    # Light mode for testing
    light_mode: bool = True     # Set to False for full grid
    light_grid_size: int = 5    # Grid subset size when light_mode=True

    # Enabled horizons for experiments
    enabled_horizons: List[int] = field(default_factory=lambda: [1, 3, 6, 12])

    # Branch 1: ConvLSTM configuration (V2 Enhanced)
    convlstm_filters: List[int] = field(default_factory=lambda: [32, 16])
    convlstm_kernel_size: int = 3
    convlstm_attention: bool = True
    convlstm_bidirectional: bool = True
    convlstm_residual: bool = True
    convlstm_output_dim: int = 64

    # Branch 2: GNN-TAT configuration (V4)
    gnn_type: str = 'GAT'           # GAT, SAGE, or GCN
    gnn_hidden_dim: int = 64
    gnn_num_layers: int = 2
    gnn_num_heads: int = 4          # For GAT
    gnn_temporal_heads: int = 4
    gnn_lstm_hidden: int = 64
    gnn_lstm_layers: int = 2
    gnn_output_dim: int = 64
    gnn_dropout: float = 0.1

    # Graph construction
    edge_threshold: float = 0.3
    max_neighbors: int = 8
    use_distance_edges: bool = True
    use_elevation_edges: bool = True
    use_correlation_edges: bool = True
    distance_scale_km: float = 10.0
    elevation_weight: float = 0.3
    correlation_weight: float = 0.5
    elevation_scale_m: float = 500.0
    min_edge_weight: float = 0.01

    # Grid-Graph Fusion
    fusion_type: str = 'auto'       # cross_attention, gated, or auto
    fusion_heads: int = 4
    fusion_hidden_dim: int = 64
    fusion_dropout: float = 0.1
    fusion_max_nodes: int = 1024

    # Meta-Learner
    meta_hidden_dim: int = 128
    meta_dropout: float = 0.1
    meta_use_context_features: bool = True
    meta_context_features: List[str] = field(
        default_factory=lambda: ['mean_elevation', 'elevation_cluster', 'temporal_regime']
    )

    # Training
    epochs: int = 200
    batch_size: int = 4
    learning_rate: float = 0.0005
    weight_decay: float = 1e-4
    patience: int = 60
    gradient_clip: float = 1.0

# Initialize config
CONFIG = V5Config()
print("V5 Configuration initialized:")
print(f"  - Light mode: {CONFIG.light_mode}")
print(f"  - GNN type: {CONFIG.gnn_type}")
print(f"  - Enabled horizons: {CONFIG.enabled_horizons}")
print(f"  - Epochs: {CONFIG.epochs}")
print(f"  - Fusion type: {CONFIG.fusion_type}")


V5 Configuration initialized:
  - Light mode: True
  - GNN type: GAT
  - Enabled horizons: [1, 3, 6, 12]
  - Epochs: 200


## 3. Data Loading and Validation

In [4]:
# =============================================================================
# SECTION 3: DATA LOADING AND FEATURE EXTRACTION
# =============================================================================

# Feature set definitions (from CLAUDE.md Section 8)
FEATURE_SETS = {
    'BASIC': [
        'year', 'month', 'month_sin', 'month_cos', 'doy_sin', 'doy_cos',
        'max_daily_precipitation', 'min_daily_precipitation', 'daily_precipitation_std',
        'elevation', 'slope', 'aspect'
    ],
    'KCE': [
        'year', 'month', 'month_sin', 'month_cos', 'doy_sin', 'doy_cos',
        'max_daily_precipitation', 'min_daily_precipitation', 'daily_precipitation_std',
        'elevation', 'slope', 'aspect',
        'elev_high', 'elev_med', 'elev_low'
    ],
    'PAFC': [
        'year', 'month', 'month_sin', 'month_cos', 'doy_sin', 'doy_cos',
        'max_daily_precipitation', 'min_daily_precipitation', 'daily_precipitation_std',
        'elevation', 'slope', 'aspect',
        'elev_high', 'elev_med', 'elev_low',
        'total_precipitation_lag1', 'total_precipitation_lag2', 'total_precipitation_lag12'
    ]
}

def load_dataset(data_path: Path, config: V5Config) -> xr.Dataset:
    """Load and validate the NetCDF dataset."""
    print(f"Loading dataset from: {data_path}")
    ds = xr.open_dataset(data_path)
    
    # Print dataset info
    print(f"\nDataset dimensions:")
    for dim, size in ds.dims.items():
        print(f"  - {dim}: {size}")
    
    print(f"\nAvailable variables: {list(ds.data_vars)}")
    
    # Apply light mode if enabled
    if config.light_mode:
        lat_slice = slice(0, config.light_grid_size)
        lon_slice = slice(0, config.light_grid_size)
        ds = ds.isel(lat=lat_slice, lon=lon_slice)
        print(f"\nLight mode enabled: using {config.light_grid_size}x{config.light_grid_size} grid")
    
    return ds

def create_elevation_clusters(ds: xr.Dataset, n_clusters: int = 3) -> xr.Dataset:
    """Add elevation cluster features (KCE) to dataset."""
    elevation = ds['elevation'].values
    valid_mask = ~np.isnan(elevation)
    
    # Flatten for clustering
    elev_flat = elevation[valid_mask].reshape(-1, 1)
    
    # K-means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=SEED, n_init=10)
    labels = np.full(elevation.shape, -1)
    labels[valid_mask] = kmeans.fit_predict(elev_flat)
    
    # Create one-hot encoded features
    for i, name in enumerate(['elev_low', 'elev_med', 'elev_high']):
        cluster_data = np.zeros_like(elevation)
        cluster_data[labels == i] = 1.0
        ds[name] = xr.DataArray(
            data=cluster_data,
            dims=['lat', 'lon'],
            attrs={'description': f'Elevation cluster {name}'}
        )
    
    print(f"Added elevation clusters: elev_low, elev_med, elev_high")
    return ds

def extract_features(ds: xr.Dataset, feature_names: List[str]) -> np.ndarray:
    """Extract features from dataset into numpy array."""
    features = []
    
    for name in feature_names:
        if name in ds.data_vars:
            data = ds[name].values
            # Handle different dimensions
            if data.ndim == 2:  # (lat, lon) - static features
                # Broadcast to (time, lat, lon)
                data = np.broadcast_to(data, (ds.dims['time'], *data.shape))
            features.append(data)
        else:
            print(f"Warning: Feature '{name}' not found in dataset")
    
    # Stack features: (time, lat, lon, n_features)
    features = np.stack(features, axis=-1)
    print(f"Extracted features shape: {features.shape}")
    
    return features.astype(np.float32)

# Load dataset
ds = load_dataset(DATA_FILE, CONFIG)
ds = create_elevation_clusters(ds)

Loading dataset from: d:\github.com\ninja-marduk\ml_precipitation_prediction\data\processed\dataset_monthly_boyaca.nc


FileNotFoundError: [Errno 2] No such file or directory: 'd:\\github.com\\ninja-marduk\\ml_precipitation_prediction\\data\\processed\\dataset_monthly_boyaca.nc'

## 4. Graph Construction for GNN Branch

In [ ]:
# =============================================================================
# SECTION 4: GRAPH CONSTRUCTION
# =============================================================================

class SpatialGraphBuilder:
    """Build spatial graph for GNN branch based on geographic and topographic similarity."""

    def __init__(self, config: V5Config):
        self.config = config

    @staticmethod
    def _safe_correlation(ts_a: np.ndarray, ts_b: np.ndarray) -> float:
        """Compute correlation robustly (returns 0.0 for invalid cases)."""
        mask = np.isfinite(ts_a) & np.isfinite(ts_b)
        if mask.sum() < 2:
            return 0.0
        a = ts_a[mask]
        b = ts_b[mask]
        a = a - a.mean()
        b = b - b.mean()
        denom = np.sqrt(np.sum(a * a)) * np.sqrt(np.sum(b * b))
        if denom < 1e-6:
            return 0.0
        corr = float(np.sum(a * b) / denom)
        if not np.isfinite(corr):
            return 0.0
        return corr

    def build_graph(self, ds: xr.Dataset) -> Tuple[torch.Tensor, torch.Tensor]:
        """Build graph structure from dataset.

        Returns:
            edge_index: (2, num_edges) tensor of edge indices
            edge_weight: (num_edges,) tensor of edge weights
        """
        lat = ds['lat'].values
        lon = ds['lon'].values
        elevation = ds['elevation'].values

        n_lat, n_lon = len(lat), len(lon)
        n_nodes = n_lat * n_lon

        print(f"Building graph for {n_lat}x{n_lon} = {n_nodes} nodes")

        # Create node positions in km for distance scaling
        lat_grid, lon_grid = np.meshgrid(lat, lon, indexing='ij')
        mean_lat = float(np.mean(lat))
        km_per_deg_lat = 111.32
        km_per_deg_lon = 111.32 * np.cos(np.deg2rad(mean_lat))
        positions = np.stack(
            [lat_grid.flatten() * km_per_deg_lat, lon_grid.flatten() * km_per_deg_lon],
            axis=1
        )
        elev_flat = elevation.flatten()

        # Precompute precipitation time series for correlation edges
        precip_flat = None
        if self.config.use_correlation_edges:
            precip = ds['total_precipitation'].values.astype(np.float32)
            precip_flat = precip.reshape(precip.shape[0], n_nodes)

        edges = []
        weights = []

        distance_coeff = 1.0
        if self.config.use_elevation_edges or self.config.use_correlation_edges:
            distance_coeff = max(
                0.0,
                1.0 - self.config.elevation_weight - self.config.correlation_weight
            )

        for i in range(n_nodes):
            # Calculate distances to all other nodes
            distances = np.sqrt(np.sum((positions - positions[i])**2, axis=1))

            # Get k nearest neighbors (excluding self)
            distances[i] = np.inf
            nearest_idx = np.argsort(distances)[:self.config.max_neighbors]
            ts_i = precip_flat[:, i] if precip_flat is not None else None

            for j in nearest_idx:
                if distances[j] == np.inf:
                    continue

                component_weights = []

                if self.config.use_distance_edges:
                    dist_weight = np.exp(-distances[j] / self.config.distance_scale_km)
                    component_weights.append((dist_weight, distance_coeff))

                if self.config.use_elevation_edges:
                    if not np.isnan(elev_flat[i]) and not np.isnan(elev_flat[j]):
                        elev_diff = np.abs(elev_flat[i] - elev_flat[j])
                        elev_weight = np.exp(-elev_diff / self.config.elevation_scale_m)
                    else:
                        elev_weight = 0.5
                    component_weights.append((elev_weight, self.config.elevation_weight))

                if self.config.use_correlation_edges:
                    if precip_flat is not None:
                        corr = self._safe_correlation(ts_i, precip_flat[:, j])
                    else:
                        corr = 0.0
                    corr_weight = max(0.0, corr)
                    component_weights.append((corr_weight, self.config.correlation_weight))

                if not component_weights:
                    continue

                coeff_sum = sum(weight for _, weight in component_weights)
                if coeff_sum <= 0:
                    coeff_sum = len(component_weights)
                    combined_weight = sum(val for val, _ in component_weights) / coeff_sum
                else:
                    combined_weight = sum(val * weight for val, weight in component_weights) / coeff_sum

                threshold = max(self.config.edge_threshold, self.config.min_edge_weight)
                if combined_weight >= threshold:
                    edges.append([i, j])
                    weights.append(combined_weight)

        edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
        edge_weight = torch.tensor(weights, dtype=torch.float32)

        print(f"Graph built: {n_nodes} nodes, {edge_index.shape[1]} edges")
        print(f"Average edges per node: {edge_index.shape[1] / n_nodes:.1f}")

        return edge_index, edge_weight

# Build graph
graph_builder = SpatialGraphBuilder(CONFIG)
edge_index, edge_weight = graph_builder.build_graph(ds)
print(f"Edge index shape: {edge_index.shape}")
print(f"Edge weight shape: {edge_weight.shape}")


## 5. Data Preprocessing Pipeline

In [ ]:
# =============================================================================
# SECTION 5: DATA PREPROCESSING AND DATASET CLASS
# =============================================================================

def create_temporal_windows(
    features: np.ndarray,
    target: np.ndarray,
    input_window: int,
    horizon: int
) -> Tuple[np.ndarray, np.ndarray]:
    """Create temporal windows for sequence-to-sequence prediction.

    Args:
        features: (time, lat, lon, n_features)
        target: (time, lat, lon)
        input_window: Number of input timesteps
        horizon: Number of output timesteps

    Returns:
        X: (n_samples, input_window, lat, lon, n_features)
        Y: (n_samples, horizon, lat, lon)
    """
    n_time = features.shape[0]
    n_samples = n_time - input_window - horizon + 1

    if n_samples <= 0:
        raise ValueError(f"Not enough timesteps: {n_time} < {input_window + horizon}")

    X_list = []
    Y_list = []

    for i in range(n_samples):
        X_list.append(features[i:i+input_window])
        Y_list.append(target[i+input_window:i+input_window+horizon])

    X = np.stack(X_list, axis=0)
    Y = np.stack(Y_list, axis=0)

    print(f"Created {n_samples} samples")
    print(f"  X shape: {X.shape}")
    print(f"  Y shape: {Y.shape}")

    return X, Y

class V5DualBranchDataset(Dataset):
    """Dataset for V5 dual-branch model (ConvLSTM + GNN).

    Provides both grid (for ConvLSTM) and graph (for GNN) representations.
    """

    def __init__(
        self,
        X_basic: np.ndarray,    # BASIC features for ConvLSTM
        X_kce: np.ndarray,      # KCE features for GNN
        Y: np.ndarray,
        edge_index: torch.Tensor,
        edge_weight: torch.Tensor
    ):
        self.X_basic = torch.tensor(X_basic, dtype=torch.float32)
        self.X_kce = torch.tensor(X_kce, dtype=torch.float32)
        self.Y = torch.tensor(Y, dtype=torch.float32)
        self.edge_index = edge_index
        self.edge_weight = edge_weight

        # Grid dimensions
        self.n_lat = X_basic.shape[2]
        self.n_lon = X_basic.shape[3]
        self.n_nodes = self.n_lat * self.n_lon

    def __len__(self):
        return len(self.X_basic)

    def __getitem__(self, idx):
        # Grid format for ConvLSTM: (seq, lat, lon, features)
        x_grid = self.X_basic[idx]

        # Graph format for GNN: (seq, n_nodes, features)
        x_graph = self.X_kce[idx].reshape(
            self.X_kce.shape[1], self.n_nodes, -1
        )

        # Target: (horizon, lat, lon)
        y = self.Y[idx]

        return {
            'x_grid': x_grid,           # For ConvLSTM branch
            'x_graph': x_graph,         # For GNN branch
            'y': y,
            'edge_index': self.edge_index,
            'edge_weight': self.edge_weight
        }

def prepare_data(ds: xr.Dataset, config: V5Config, edge_index: torch.Tensor, edge_weight: torch.Tensor):
    """Prepare data for training."""
    print("\n" + "="*60)
    print("Preparing data for V5 dual-branch model")
    print("="*60)

    # Extract features for both branches
    print("\nExtracting BASIC features (ConvLSTM branch):")
    features_basic = extract_features(ds, FEATURE_SETS['BASIC'])

    print("\nExtracting KCE features (GNN branch):")
    features_kce = extract_features(ds, FEATURE_SETS['KCE'])

    # Target variable
    target = ds['total_precipitation'].values.astype(np.float32)
    print(f"\nTarget shape: {target.shape}")

    # Handle NaN values
    features_basic = np.nan_to_num(features_basic, nan=0.0)
    features_kce = np.nan_to_num(features_kce, nan=0.0)
    target = np.nan_to_num(target, nan=0.0)

    # Create temporal windows (raw)
    print("\nCreating temporal windows:")
    X_basic_raw, Y = create_temporal_windows(
        features_basic, target,
        config.input_window, config.horizon
    )
    X_kce_raw, _ = create_temporal_windows(
        features_kce, target,
        config.input_window, config.horizon
    )

    # Train/val split
    n_train = int(len(X_basic_raw) * config.train_val_split)

    # Normalize using training split only (avoid leakage)
    print("\nNormalizing features using training split only:")
    X_basic_train = X_basic_raw[:n_train]
    X_kce_train = X_kce_raw[:n_train]

    basic_mean = X_basic_train.mean(axis=(0, 1, 2, 3), keepdims=True)
    basic_std = X_basic_train.std(axis=(0, 1, 2, 3), keepdims=True)
    basic_std = np.where(basic_std > 1e-6, basic_std, 1.0)

    kce_mean = X_kce_train.mean(axis=(0, 1, 2, 3), keepdims=True)
    kce_std = X_kce_train.std(axis=(0, 1, 2, 3), keepdims=True)
    kce_std = np.where(kce_std > 1e-6, kce_std, 1.0)

    X_basic = ((X_basic_raw - basic_mean) / basic_std).astype(np.float32)
    X_kce = ((X_kce_raw - kce_mean) / kce_std).astype(np.float32)

    train_dataset = V5DualBranchDataset(
        X_basic[:n_train], X_kce[:n_train], Y[:n_train],
        edge_index, edge_weight
    )
    val_dataset = V5DualBranchDataset(
        X_basic[n_train:], X_kce[n_train:], Y[n_train:],
        edge_index, edge_weight
    )

    print(f"\nDataset split:")
    print(f"  Training samples: {len(train_dataset)}")
    print(f"  Validation samples: {len(val_dataset)}")

    return train_dataset, val_dataset, features_basic.shape[-1], features_kce.shape[-1]

# Prepare data
train_dataset, val_dataset, n_features_basic, n_features_kce = prepare_data(
    ds, CONFIG, edge_index, edge_weight
)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=CONFIG.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CONFIG.batch_size, shuffle=False)

print(f"\nData loaders created:")
print(f"  Train batches: {len(train_loader)}")
print(f"  Val batches: {len(val_loader)}")


## 6. V5 Model Architecture

### 6.1 Branch 1: ConvLSTM (Euclidean Spatial Encoder)

In [ ]:
# =============================================================================
# SECTION 6.1: CONVLSTM BRANCH (V2 Enhanced Architecture)
# =============================================================================

class ConvLSTMCell(nn.Module):
    """Single ConvLSTM cell with spatial convolutions."""

    def __init__(self, input_dim: int, hidden_dim: int, kernel_size: int):
        super().__init__()
        self.hidden_dim = hidden_dim
        padding = kernel_size // 2

        self.conv = nn.Conv2d(
            in_channels=input_dim + hidden_dim,
            out_channels=4 * hidden_dim,  # i, f, o, g gates
            kernel_size=kernel_size,
            padding=padding,
            bias=True
        )

    def forward(self, x: torch.Tensor, state: Tuple[torch.Tensor, torch.Tensor]):
        h, c = state
        combined = torch.cat([x, h], dim=1)
        gates = self.conv(combined)

        i, f, o, g = torch.split(gates, self.hidden_dim, dim=1)
        i = torch.sigmoid(i)
        f = torch.sigmoid(f)
        o = torch.sigmoid(o)
        g = torch.tanh(g)

        c_new = f * c + i * g
        h_new = o * torch.tanh(c_new)

        return h_new, c_new

    def init_hidden(self, batch_size: int, height: int, width: int, device: torch.device):
        return (
            torch.zeros(batch_size, self.hidden_dim, height, width, device=device),
            torch.zeros(batch_size, self.hidden_dim, height, width, device=device)
        )

class SpatialAttention(nn.Module):
    """Spatial attention mechanism for ConvLSTM output."""

    def __init__(self, in_channels: int):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, 1, kernel_size=1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        attention = torch.sigmoid(self.conv(x))
        return x * attention

class ConvLSTMBranch(nn.Module):
    """Branch 1: ConvLSTM encoder for Euclidean spatial patterns.

    Based on V2 Enhanced architecture with attention and residual connections.
    """

    def __init__(self, config: V5Config, n_features: int):
        super().__init__()
        self.config = config

        # Input projection
        self.input_proj = nn.Conv2d(
            n_features, config.convlstm_filters[0],
            kernel_size=1
        )

        # ConvLSTM layers
        self.convlstm_cells_fw = nn.ModuleList()
        self.convlstm_cells_bw = nn.ModuleList() if config.convlstm_bidirectional else None
        in_dim = config.convlstm_filters[0]
        for out_dim in config.convlstm_filters:
            self.convlstm_cells_fw.append(
                ConvLSTMCell(in_dim, out_dim, config.convlstm_kernel_size)
            )
            if config.convlstm_bidirectional:
                self.convlstm_cells_bw.append(
                    ConvLSTMCell(in_dim, out_dim, config.convlstm_kernel_size)
                )
            in_dim = out_dim

        # Spatial attention
        attn_in_dim = config.convlstm_filters[-1] * (2 if config.convlstm_bidirectional else 1)
        if config.convlstm_attention:
            self.attention = SpatialAttention(attn_in_dim)
        else:
            self.attention = None

        # Output projection to match GNN branch
        self.output_proj = nn.Conv2d(
            config.convlstm_filters[-1] * (2 if config.convlstm_bidirectional else 1),
            config.convlstm_output_dim,
            kernel_size=1
        )

        # Residual connection
        if config.convlstm_residual:
            self.residual_proj = nn.Conv2d(n_features, config.convlstm_output_dim, kernel_size=1)
        else:
            self.residual_proj = None

    def _run_convlstm(self, x: torch.Tensor, cells: nn.ModuleList) -> torch.Tensor:
        """Run ConvLSTM stack and return last hidden state."""
        batch_size, seq_len, _, h, w = x.shape
        layer_input = x
        for layer_idx, cell in enumerate(cells):
            h_state, c_state = cell.init_hidden(batch_size, h, w, x.device)
            outputs = []
            for t in range(seq_len):
                x_t = layer_input[:, t]
                if layer_idx == 0:
                    x_t = self.input_proj(x_t)
                h_state, c_state = cell(x_t, (h_state, c_state))
                outputs.append(h_state)
            layer_input = torch.stack(outputs, dim=1)
        return h_state

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: (batch, seq_len, lat, lon, features)

        Returns:
            output: (batch, lat, lon, output_dim)
        """
        batch_size, seq_len, h, w, _ = x.shape
        device = x.device

        # Reshape for Conv2d: (batch, seq, features, h, w)
        x_reshaped = x.permute(0, 1, 4, 2, 3)

        # Store residual
        if self.residual_proj is not None:
            residual = self.residual_proj(x_reshaped[:, -1])  # Use last timestep

        # Forward direction
        h_forward = self._run_convlstm(x_reshaped, self.convlstm_cells_fw)

        if self.config.convlstm_bidirectional:
            # Backward direction
            x_backward = x_reshaped.flip(1)
            h_backward = self._run_convlstm(x_backward, self.convlstm_cells_bw)
            output = torch.cat([h_forward, h_backward], dim=1)
        else:
            output = h_forward

        # Apply attention
        if self.attention is not None:
            output = self.attention(output)

        # Project to output dimension
        output = self.output_proj(output)  # (batch, output_dim, h, w)

        # Add residual
        if self.residual_proj is not None:
            output = output + residual

        # Reshape to (batch, h, w, output_dim)
        output = output.permute(0, 2, 3, 1)

        return output

print("ConvLSTM Branch defined successfully")


### 6.2 Branch 2: GNN-TAT (Non-Euclidean Spatial Encoder)

In [ ]:
# =============================================================================
# SECTION 6.2: GNN-TAT BRANCH (V4 Architecture)
# =============================================================================

class TemporalAttention(nn.Module):
    """Multi-head temporal self-attention."""
    
    def __init__(self, hidden_dim: int, num_heads: int, dropout: float = 0.1):
        super().__init__()
        self.attention = nn.MultiheadAttention(
            embed_dim=hidden_dim,
            num_heads=num_heads,
            dropout=dropout,
            batch_first=True
        )
        self.norm = nn.LayerNorm(hidden_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """x: (batch, seq_len, hidden_dim)"""
        attended, _ = self.attention(x, x, x)
        x = self.norm(x + self.dropout(attended))
        return x

class SpatialGNNEncoder(nn.Module):
    """GNN encoder for non-Euclidean spatial patterns."""
    
    def __init__(self, config: V5Config, n_features: int):
        super().__init__()
        self.config = config
        
        # Input projection
        self.input_proj = nn.Linear(n_features, config.gnn_hidden_dim)
        
        # GNN layers
        self.gnn_layers = nn.ModuleList()
        for i in range(config.gnn_num_layers):
            if config.gnn_type == 'GAT':
                self.gnn_layers.append(
                    GATConv(
                        config.gnn_hidden_dim,
                        config.gnn_hidden_dim // config.gnn_num_heads,
                        heads=config.gnn_num_heads,
                        dropout=config.gnn_dropout,
                        concat=True
                    )
                )
            elif config.gnn_type == 'SAGE':
                self.gnn_layers.append(
                    SAGEConv(config.gnn_hidden_dim, config.gnn_hidden_dim)
                )
            else:  # GCN
                self.gnn_layers.append(
                    GCNConv(config.gnn_hidden_dim, config.gnn_hidden_dim)
                )
        
        self.dropout = nn.Dropout(config.gnn_dropout)
        self.norm = nn.LayerNorm(config.gnn_hidden_dim)
        
    def forward(self, x: torch.Tensor, edge_index: torch.Tensor, edge_weight: torch.Tensor = None) -> torch.Tensor:
        """
        Args:
            x: (n_nodes, n_features)
            edge_index: (2, n_edges)
            edge_weight: (n_edges,)
        
        Returns:
            x: (n_nodes, hidden_dim)
        """
        x = self.input_proj(x)
        
        for gnn in self.gnn_layers:
            if self.config.gnn_type == 'GCN' and edge_weight is not None:
                x_new = gnn(x, edge_index, edge_weight)
            else:
                x_new = gnn(x, edge_index)
            x_new = F.relu(x_new)
            x_new = self.dropout(x_new)
            x = self.norm(x + x_new)  # Residual
        
        return x

class GNNTATBranch(nn.Module):
    """Branch 2: GNN-TAT encoder for non-Euclidean spatial patterns.
    
    Based on V4 architecture: GNN + Temporal Attention + LSTM.
    """
    
    def __init__(self, config: V5Config, n_features: int, n_nodes: int):
        super().__init__()
        self.config = config
        self.n_nodes = n_nodes
        
        # Spatial GNN encoder
        self.spatial_encoder = SpatialGNNEncoder(config, n_features)
        
        # Temporal attention
        self.temporal_attention = TemporalAttention(
            config.gnn_hidden_dim,
            config.gnn_temporal_heads,
            config.gnn_dropout
        )
        
        # Bidirectional LSTM
        self.lstm = nn.LSTM(
            input_size=config.gnn_hidden_dim,
            hidden_size=config.gnn_lstm_hidden,
            num_layers=config.gnn_lstm_layers,
            batch_first=True,
            bidirectional=True,
            dropout=config.gnn_dropout if config.gnn_lstm_layers > 1 else 0
        )
        
        # Output projection
        self.output_proj = nn.Linear(
            config.gnn_lstm_hidden * 2,  # Bidirectional
            config.gnn_output_dim
        )
        
    def forward(
        self,
        x: torch.Tensor,
        edge_index: torch.Tensor,
        edge_weight: torch.Tensor = None
    ) -> torch.Tensor:
        """
        Args:
            x: (batch, seq_len, n_nodes, n_features)
            edge_index: (2, n_edges)
            edge_weight: (n_edges,)
        
        Returns:
            output: (batch, n_nodes, output_dim)
        """
        batch_size, seq_len, n_nodes, _ = x.shape
        
        # Process each timestep through GNN
        spatial_outputs = []
        for t in range(seq_len):
            # Process all nodes for this timestep
            x_t = x[:, t]  # (batch, n_nodes, features)
            
            # Flatten batch and process
            batch_outputs = []
            for b in range(batch_size):
                h = self.spatial_encoder(x_t[b], edge_index, edge_weight)
                batch_outputs.append(h)
            
            spatial_out = torch.stack(batch_outputs, dim=0)  # (batch, n_nodes, hidden)
            spatial_outputs.append(spatial_out)
        
        # Stack temporal: (batch, seq_len, n_nodes, hidden)
        spatial_outputs = torch.stack(spatial_outputs, dim=1)
        
        # Apply temporal attention per node
        # Reshape: (batch*n_nodes, seq_len, hidden)
        x_temporal = spatial_outputs.permute(0, 2, 1, 3).reshape(
            batch_size * n_nodes, seq_len, -1
        )
        x_temporal = self.temporal_attention(x_temporal)
        
        # LSTM processing
        lstm_out, _ = self.lstm(x_temporal)  # (batch*n_nodes, seq_len, hidden*2)
        
        # Take last timestep
        last_hidden = lstm_out[:, -1]  # (batch*n_nodes, hidden*2)
        
        # Project to output dimension
        output = self.output_proj(last_hidden)  # (batch*n_nodes, output_dim)
        
        # Reshape back: (batch, n_nodes, output_dim)
        output = output.reshape(batch_size, n_nodes, -1)
        
        return output

print("GNN-TAT Branch defined successfully")

### 6.3 Grid-Graph Fusion Module

In [ ]:
# =============================================================================
# SECTION 6.3: GRID-GRAPH FUSION MODULE (NOVEL INNOVATION)
# =============================================================================

class GridGraphFusion(nn.Module):
    """Novel Grid-Graph Fusion via Cross-Attention.

    Bridges Euclidean (ConvLSTM grid) and Non-Euclidean (GNN graph) representations.
    This is a key innovation in the V5 architecture.
    """

    def __init__(self, config: V5Config, n_lat: int, n_lon: int):
        super().__init__()
        self.config = config
        self.n_lat = n_lat
        self.n_lon = n_lon
        self.n_nodes = n_lat * n_lon
        self._warned_large = False

        hidden_dim = config.fusion_hidden_dim

        # Project both branches to same dimension
        self.grid_proj = nn.Linear(config.convlstm_output_dim, hidden_dim)
        self.graph_proj = nn.Linear(config.gnn_output_dim, hidden_dim)

        # Cross-attention: grid attends to graph
        self.grid_to_graph_attn = nn.MultiheadAttention(
            embed_dim=hidden_dim,
            num_heads=config.fusion_heads,
            dropout=config.fusion_dropout,
            batch_first=True
        )

        # Cross-attention: graph attends to grid
        self.graph_to_grid_attn = nn.MultiheadAttention(
            embed_dim=hidden_dim,
            num_heads=config.fusion_heads,
            dropout=config.fusion_dropout,
            batch_first=True
        )

        # Gated fusion (lighter fallback)
        self.gate_mlp = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(config.fusion_dropout),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )

        # Layer norms
        self.norm_grid = nn.LayerNorm(hidden_dim)
        self.norm_graph = nn.LayerNorm(hidden_dim)

        # Final fusion
        self.fusion_mlp = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(config.fusion_dropout),
            nn.Linear(hidden_dim, hidden_dim)
        )

    def _select_fusion_mode(self) -> str:
        if self.config.fusion_type == 'auto':
            if self.n_nodes > self.config.fusion_max_nodes:
                return 'gated'
            return 'cross_attention'
        return self.config.fusion_type

    def forward(
        self,
        grid_features: torch.Tensor,   # From ConvLSTM: (batch, lat, lon, dim)
        graph_features: torch.Tensor   # From GNN: (batch, n_nodes, dim)
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Fuse grid and graph representations through cross-attention.

        Returns:
            fused_grid: (batch, lat, lon, hidden_dim)
            fused_graph: (batch, n_nodes, hidden_dim)
        """
        batch_size = grid_features.shape[0]

        # Flatten grid to sequence: (batch, n_nodes, dim)
        grid_flat = grid_features.reshape(batch_size, self.n_nodes, -1)

        # Project to common dimension
        grid_proj = self.grid_proj(grid_flat)
        graph_proj = self.graph_proj(graph_features)

        mode = self._select_fusion_mode()
        if mode == 'cross_attention':
            if self.n_nodes > self.config.fusion_max_nodes and not self._warned_large:
                print("Warning: cross-attention on large grids may OOM. Consider fusion_type='gated'.")
                self._warned_large = True

            # Cross-attention: grid attends to graph
            grid_attended, _ = self.grid_to_graph_attn(
                query=grid_proj,
                key=graph_proj,
                value=graph_proj
            )
            grid_fused = self.norm_grid(grid_proj + grid_attended)

            # Cross-attention: graph attends to grid
            graph_attended, _ = self.graph_to_grid_attn(
                query=graph_proj,
                key=grid_proj,
                value=grid_proj
            )
            graph_fused = self.norm_graph(graph_proj + graph_attended)
        elif mode == 'gated':
            gate_input = torch.cat([grid_proj, graph_proj], dim=-1)
            gate = self.gate_mlp(gate_input)
            grid_fused = self.norm_grid(gate * grid_proj)
            graph_fused = self.norm_graph((1.0 - gate) * graph_proj)
        else:
            grid_fused = self.norm_grid(grid_proj)
            graph_fused = self.norm_graph(graph_proj)

        # Combine fused features
        combined = torch.cat([grid_fused, graph_fused], dim=-1)
        fused = self.fusion_mlp(combined)

        # Reshape grid back to spatial
        fused_grid = fused.reshape(batch_size, self.n_lat, self.n_lon, -1)
        fused_graph = fused  # Keep as (batch, n_nodes, hidden_dim)

        return fused_grid, fused_graph

print("Grid-Graph Fusion Module defined successfully")


### 6.4 Meta-Learner (Interpretable Weighted Fusion)

In [ ]:
# =============================================================================
# SECTION 6.4: META-LEARNER (INTERPRETABLE BRANCH WEIGHTING)
# =============================================================================

class MetaLearner(nn.Module):
    """Interpretable Meta-Learner for branch weighting.

    Learns context-dependent weights for ConvLSTM and GNN predictions:
        output = w1(context) * ConvLSTM + w2(context) * GNN

    Enables analysis of which branch contributes more for different contexts
    (elevation regimes, temporal patterns, etc.).
    """

    def __init__(self, config: V5Config, n_lat: int, n_lon: int, horizon: int, context_dim: int = 0):
        super().__init__()
        self.config = config
        self.n_lat = n_lat
        self.n_lon = n_lon
        self.n_nodes = n_lat * n_lon
        self.horizon = horizon
        self.context_dim = context_dim

        hidden_dim = config.fusion_hidden_dim
        meta_dim = config.meta_hidden_dim

        # Prediction heads for each branch
        self.convlstm_head = nn.Sequential(
            nn.Linear(hidden_dim, meta_dim),
            nn.ReLU(),
            nn.Dropout(config.meta_dropout),
            nn.Linear(meta_dim, horizon)
        )

        self.gnn_head = nn.Sequential(
            nn.Linear(hidden_dim, meta_dim),
            nn.ReLU(),
            nn.Dropout(config.meta_dropout),
            nn.Linear(meta_dim, horizon)
        )

        # Context-dependent weight network
        # Input: concatenated fused features (+ optional context)
        self.weight_network = nn.Sequential(
            nn.Linear(hidden_dim * 2 + context_dim, meta_dim),
            nn.ReLU(),
            nn.Dropout(config.meta_dropout),
            nn.Linear(meta_dim, 2),  # 2 weights: w_convlstm, w_gnn
            nn.Softmax(dim=-1)       # Ensure weights sum to 1
        )

    def forward(
        self,
        fused_grid: torch.Tensor,    # (batch, lat, lon, hidden_dim)
        fused_graph: torch.Tensor,   # (batch, n_nodes, hidden_dim)
        context_features: Optional[torch.Tensor] = None  # (batch, n_nodes, context_dim)
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Generate predictions with interpretable branch weighting.

        Returns:
            predictions: (batch, horizon, lat, lon)
            weights: (batch, n_nodes, 2) - weights for each branch per location
        """
        batch_size = fused_grid.shape[0]

        # Flatten grid to match graph
        grid_flat = fused_grid.reshape(batch_size, self.n_nodes, -1)

        # Generate predictions from each branch
        pred_convlstm = self.convlstm_head(grid_flat)   # (batch, n_nodes, horizon)
        pred_gnn = self.gnn_head(fused_graph)          # (batch, n_nodes, horizon)

        # Compute context-dependent weights
        # Context is the concatenation of both fused representations
        if context_features is not None:
            context = torch.cat([grid_flat, fused_graph, context_features], dim=-1)
        else:
            context = torch.cat([grid_flat, fused_graph], dim=-1)
        weights = self.weight_network(context)  # (batch, n_nodes, 2)

        # Weighted combination
        w_convlstm = weights[..., 0:1]  # (batch, n_nodes, 1)
        w_gnn = weights[..., 1:2]       # (batch, n_nodes, 1)

        predictions = w_convlstm * pred_convlstm + w_gnn * pred_gnn

        # Reshape to spatial grid: (batch, horizon, lat, lon)
        predictions = predictions.reshape(batch_size, self.n_lat, self.n_lon, self.horizon)
        predictions = predictions.permute(0, 3, 1, 2)  # (batch, horizon, lat, lon)

        return predictions, weights

print("Meta-Learner defined successfully")


### 6.5 Complete V5 Stacking Model

In [ ]:
# =============================================================================
# SECTION 6.5: COMPLETE V5 STACKING MODEL
# =============================================================================

class V5StackingModel(nn.Module):
    """V5 GNN-ConvLSTM Stacking Ensemble.

    Novel hybrid architecture combining:
    - Branch 1: ConvLSTM for Euclidean spatial patterns (BASIC features)
    - Branch 2: GNN-TAT for non-Euclidean topographic relations (KCE features)
    - Grid-Graph Fusion via cross-attention
    - Interpretable Meta-Learner for weighted combination

    Innovation: First work to combine GNN and ConvLSTM in stacking ensemble
    for precipitation prediction (verified January 2026).
    """

    def __init__(
        self,
        config: V5Config,
        n_features_basic: int,
        n_features_kce: int,
        n_lat: int,
        n_lon: int,
        horizon: int
    ):
        super().__init__()
        self.config = config
        self.n_lat = n_lat
        self.n_lon = n_lon
        self.n_nodes = n_lat * n_lon
        self.horizon = horizon
        self.use_context_features = config.meta_use_context_features

        # Context feature indices (from KCE)
        self.context_indices = {}
        self.context_dim = 0
        if self.use_context_features:
            try:
                kce_features = FEATURE_SETS['KCE']
                self.context_indices = {
                    'elevation': kce_features.index('elevation'),
                    'elev_low': kce_features.index('elev_low'),
                    'elev_med': kce_features.index('elev_med'),
                    'elev_high': kce_features.index('elev_high'),
                    'month_sin': kce_features.index('month_sin'),
                    'month_cos': kce_features.index('month_cos')
                }
                # mean_elevation (1) + elevation_cluster (3) + temporal_regime (2)
                self.context_dim = 6
            except Exception as exc:
                print(f"Warning: context features unavailable ({exc}); disabling meta context.")
                self.use_context_features = False
                self.context_dim = 0

        # Branch 1: ConvLSTM (Euclidean)
        self.convlstm_branch = ConvLSTMBranch(config, n_features_basic)

        # Branch 2: GNN-TAT (Non-Euclidean)
        self.gnn_branch = GNNTATBranch(config, n_features_kce, self.n_nodes)

        # Grid-Graph Fusion
        self.fusion = GridGraphFusion(config, n_lat, n_lon)

        # Meta-Learner
        self.meta_learner = MetaLearner(config, n_lat, n_lon, horizon, context_dim=self.context_dim)

    def _extract_context(self, x_graph: torch.Tensor) -> Optional[torch.Tensor]:
        """Extract per-node context features for the meta-learner."""
        if not self.use_context_features:
            return None

        idx = self.context_indices

        # (batch, seq, n_nodes, features)
        elevation = x_graph[:, -1, :, idx['elevation']].unsqueeze(-1)
        elev_clusters = x_graph[:, -1, :, [idx['elev_low'], idx['elev_med'], idx['elev_high']]]

        # Temporal regime from mean seasonal encoding
        month_sin = x_graph[:, :, :, idx['month_sin']].mean(dim=1)
        month_cos = x_graph[:, :, :, idx['month_cos']].mean(dim=1)
        temporal = torch.stack([month_sin, month_cos], dim=-1)

        context = torch.cat([elevation, elev_clusters, temporal], dim=-1)
        return context

    def forward(
        self,
        x_grid: torch.Tensor,       # (batch, seq, lat, lon, features_basic)
        x_graph: torch.Tensor,      # (batch, seq, n_nodes, features_kce)
        edge_index: torch.Tensor,   # (2, n_edges)
        edge_weight: torch.Tensor   # (n_edges,)
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Forward pass through V5 stacking model.

        Returns:
            predictions: (batch, horizon, lat, lon) - precipitation predictions
            weights: (batch, n_nodes, 2) - branch weights for interpretability
        """
        # Branch 1: ConvLSTM processing
        convlstm_out = self.convlstm_branch(x_grid)  # (batch, lat, lon, dim)

        # Branch 2: GNN-TAT processing
        gnn_out = self.gnn_branch(x_graph, edge_index, edge_weight)  # (batch, n_nodes, dim)

        # Grid-Graph Fusion
        fused_grid, fused_graph = self.fusion(convlstm_out, gnn_out)

        # Meta-Learner for final predictions
        context = self._extract_context(x_graph)
        predictions, weights = self.meta_learner(fused_grid, fused_graph, context)

        return predictions, weights

    def count_parameters(self) -> Dict[str, int]:
        """Count parameters per component."""
        counts = {
            'convlstm_branch': sum(p.numel() for p in self.convlstm_branch.parameters()),
            'gnn_branch': sum(p.numel() for p in self.gnn_branch.parameters()),
            'fusion': sum(p.numel() for p in self.fusion.parameters()),
            'meta_learner': sum(p.numel() for p in self.meta_learner.parameters())
        }
        counts['total'] = sum(counts.values())
        return counts

# Test model instantiation
n_lat, n_lon = (CONFIG.light_grid_size, CONFIG.light_grid_size) if CONFIG.light_mode else (ds.dims['lat'], ds.dims['lon'])

model = V5StackingModel(
    config=CONFIG,
    n_features_basic=n_features_basic,
    n_features_kce=n_features_kce,
    n_lat=n_lat,
    n_lon=n_lon,
    horizon=CONFIG.horizon
).to(device)

# Print parameter counts
param_counts = model.count_parameters()
print("\nV5 Model Parameter Counts:")
print("="*40)
for name, count in param_counts.items():
    print(f"  {name}: {count:,}")
print(f"\nTotal: {param_counts['total']:,} parameters")
print(f"Target: < 200,000 parameters")


## 7. Training Infrastructure

In [ ]:
# =============================================================================
# SECTION 7: TRAINING INFRASTRUCTURE
# =============================================================================

class V5Trainer:
    """Training infrastructure for V5 model."""

    def __init__(
        self,
        model: V5StackingModel,
        config: V5Config,
        train_loader: DataLoader,
        val_loader: DataLoader,
        device: torch.device
    ):
        self.model = model
        self.config = config
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device

        # Loss function
        self.criterion = nn.MSELoss()

        # Optimizer
        self.optimizer = torch.optim.AdamW(
            model.parameters(),
            lr=config.learning_rate,
            weight_decay=config.weight_decay
        )

        # Learning rate scheduler
        self.scheduler = ReduceLROnPlateau(
            self.optimizer,
            mode='min',
            factor=0.5,
            patience=20,
            min_lr=1e-6
        )

        # Training history
        self.history = {
            'train_loss': [],
            'val_loss': [],
            'train_mae': [],
            'val_mae': [],
            'lr': [],
            'branch_weights': []  # Track meta-learner weights
        }

        # Early stopping
        self.best_val_loss = float('inf')
        self.patience_counter = 0
        self.best_epoch = 0
        self.best_model_state = None

    def train_epoch(self) -> Tuple[float, float]:
        """Train for one epoch."""
        self.model.train()
        total_loss = 0.0
        total_mae = 0.0
        n_batches = 0

        for batch in self.train_loader:
            x_grid = batch['x_grid'].to(self.device)
            x_graph = batch['x_graph'].to(self.device)
            y = batch['y'].to(self.device)
            edge_index = batch['edge_index'][0].to(self.device)
            edge_weight = batch['edge_weight'][0].to(self.device)

            self.optimizer.zero_grad()

            # Forward pass
            predictions, weights = self.model(x_grid, x_graph, edge_index, edge_weight)

            # Loss computation
            loss = self.criterion(predictions, y)

            # Backward pass
            loss.backward()

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(
                self.model.parameters(),
                self.config.gradient_clip
            )

            self.optimizer.step()

            # Metrics
            total_loss += loss.item()
            total_mae += torch.mean(torch.abs(predictions - y)).item()
            n_batches += 1

        return total_loss / n_batches, total_mae / n_batches

    @torch.no_grad()
    def validate(self) -> Tuple[float, float, np.ndarray]:
        """Validate model."""
        self.model.eval()
        total_loss = 0.0
        total_mae = 0.0
        n_batches = 0
        all_weights = []

        for batch in self.val_loader:
            x_grid = batch['x_grid'].to(self.device)
            x_graph = batch['x_graph'].to(self.device)
            y = batch['y'].to(self.device)
            edge_index = batch['edge_index'][0].to(self.device)
            edge_weight = batch['edge_weight'][0].to(self.device)

            predictions, weights = self.model(x_grid, x_graph, edge_index, edge_weight)

            loss = self.criterion(predictions, y)

            total_loss += loss.item()
            total_mae += torch.mean(torch.abs(predictions - y)).item()
            all_weights.append(weights.cpu().numpy())
            n_batches += 1

        avg_weights = np.concatenate(all_weights, axis=0).mean(axis=(0, 1))

        return total_loss / n_batches, total_mae / n_batches, avg_weights

    def train(self, output_dir: Path = None) -> Dict:
        """Full training loop."""
        print("\n" + "="*70)
        print("Starting V5 Training")
        print("="*70)

        start_time = datetime.now()

        for epoch in range(self.config.epochs):
            # Train
            train_loss, train_mae = self.train_epoch()

            # Validate
            val_loss, val_mae, weights = self.validate()

            # Learning rate scheduling
            self.scheduler.step(val_loss)
            current_lr = self.optimizer.param_groups[0]['lr']

            # Record history
            self.history['train_loss'].append(train_loss)
            self.history['val_loss'].append(val_loss)
            self.history['train_mae'].append(train_mae)
            self.history['val_mae'].append(val_mae)
            self.history['lr'].append(current_lr)
            self.history['branch_weights'].append(weights.tolist())

            # Early stopping check
            if val_loss < self.best_val_loss:
                self.best_val_loss = val_loss
                self.best_epoch = epoch
                self.patience_counter = 0
                self.best_model_state = copy.deepcopy(self.model.state_dict())

                # Save checkpoint
                if output_dir:
                    checkpoint_path = output_dir / 'v5_stacking_best.pt'
                    torch.save({
                        'epoch': epoch,
                        'model_state_dict': self.model.state_dict(),
                        'optimizer_state_dict': self.optimizer.state_dict(),
                        'val_loss': val_loss,
                        'config': asdict(self.config)
                    }, checkpoint_path)
            else:
                self.patience_counter += 1

            # Logging
            if (epoch + 1) % 10 == 0 or epoch == 0:
                w_conv, w_gnn = weights
                print(f"Epoch {epoch+1:3d}/{self.config.epochs} | "
                      f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | "
                      f"LR: {current_lr:.2e} | W_conv: {w_conv:.2f} W_gnn: {w_gnn:.2f}")

            # Early stopping
            if self.patience_counter >= self.config.patience:
                print(f"\nEarly stopping at epoch {epoch+1}")
                break

        # Restore best model
        if self.best_model_state:
            self.model.load_state_dict(self.best_model_state)

        elapsed = datetime.now() - start_time

        print(f"\nTraining completed in {elapsed}")
        print(f"Best validation loss: {self.best_val_loss:.4f} at epoch {self.best_epoch+1}")

        return self.history

print("V5 Trainer defined successfully")


In [ ]:
# =============================================================================
# SECTION 7.2: METRICS EVALUATION
# =============================================================================

@torch.no_grad()
def evaluate_model(
    model: V5StackingModel,
    data_loader: DataLoader,
    device: torch.device
) -> Dict:
    """Evaluate model and compute metrics per horizon."""
    model.eval()
    
    all_preds = []
    all_targets = []
    all_weights = []
    
    for batch in data_loader:
        x_grid = batch['x_grid'].to(device)
        x_graph = batch['x_graph'].to(device)
        y = batch['y'].to(device)
        edge_index = batch['edge_index'][0].to(device)
        edge_weight = batch['edge_weight'][0].to(device)
        
        predictions, weights = model(x_grid, x_graph, edge_index, edge_weight)
        
        all_preds.append(predictions.cpu().numpy())
        all_targets.append(y.cpu().numpy())
        all_weights.append(weights.cpu().numpy())
    
    preds = np.concatenate(all_preds, axis=0)
    targets = np.concatenate(all_targets, axis=0)
    weights = np.concatenate(all_weights, axis=0)
    
    # Compute metrics per horizon
    metrics = {}
    n_horizons = preds.shape[1]
    
    for h in range(n_horizons):
        pred_h = preds[:, h].flatten()
        target_h = targets[:, h].flatten()
        
        # Remove NaN values
        valid_mask = ~np.isnan(pred_h) & ~np.isnan(target_h)
        pred_h = pred_h[valid_mask]
        target_h = target_h[valid_mask]
        
        if len(pred_h) == 0:
            continue
        
        # RMSE
        rmse = np.sqrt(np.mean((pred_h - target_h) ** 2))
        
        # MAE
        mae = np.mean(np.abs(pred_h - target_h))
        
        # R²
        ss_res = np.sum((target_h - pred_h) ** 2)
        ss_tot = np.sum((target_h - np.mean(target_h)) ** 2)
        r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
        
        # Bias
        mean_true = np.mean(target_h)
        mean_pred = np.mean(pred_h)
        bias_mm = mean_pred - mean_true
        bias_pct = 100 * bias_mm / mean_true if mean_true != 0 else 0
        
        metrics[f'H{h+1}'] = {
            'RMSE': float(rmse),
            'MAE': float(mae),
            'R^2': float(r2),
            'Mean_True_mm': float(mean_true),
            'Mean_Pred_mm': float(mean_pred),
            'mean_bias_mm': float(bias_mm),
            'mean_bias_pct': float(bias_pct)
        }
    
    # Average branch weights
    avg_weights = weights.mean(axis=(0, 1))
    metrics['branch_weights'] = {
        'w_convlstm': float(avg_weights[0]),
        'w_gnn': float(avg_weights[1])
    }
    
    return metrics

print("Metrics evaluation functions defined")

## 8. Main Training Loop

In [ ]:
# =============================================================================
# SECTION 8: MAIN TRAINING LOOP
# =============================================================================

def run_v5_experiments(
    ds: xr.Dataset,
    config: V5Config,
    edge_index: torch.Tensor,
    edge_weight: torch.Tensor,
    output_root: Path
):
    """Run V5 experiments across multiple horizons."""
    
    all_metrics = []
    experiment_state = {
        'config': asdict(config),
        'timestamp': datetime.now().isoformat(),
        'results': {}
    }
    
    for horizon in config.enabled_horizons:
        print("\n" + "="*70)
        print(f"TRAINING V5 STACKING MODEL - HORIZON H{horizon}")
        print("="*70)
        
        # Update horizon in config
        current_config = V5Config(**{**asdict(config), 'horizon': horizon})
        
        # Prepare data
        train_dataset, val_dataset, n_basic, n_kce = prepare_data(
            ds, current_config, edge_index, edge_weight
        )
        
        train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)
        
        # Get grid dimensions
        n_lat = train_dataset.n_lat
        n_lon = train_dataset.n_lon
        
        # Initialize model
        model = V5StackingModel(
            config=current_config,
            n_features_basic=n_basic,
            n_features_kce=n_kce,
            n_lat=n_lat,
            n_lon=n_lon,
            horizon=horizon
        ).to(device)
        
        # Create output directory for this horizon
        horizon_dir = output_root / f'h{horizon}' / 'BASIC_KCE' / 'training_metrics'
        horizon_dir.mkdir(parents=True, exist_ok=True)
        
        # Train
        trainer = V5Trainer(model, current_config, train_loader, val_loader, device)
        history = trainer.train(horizon_dir)
        
        # Evaluate
        metrics = evaluate_model(model, val_loader, device)
        
        # Save training history
        history_data = {
            'model_name': 'V5_STACKING',
            'experiment': 'BASIC_KCE',
            'horizon': horizon,
            'best_epoch': trainer.best_epoch,
            'best_val_loss': trainer.best_val_loss,
            'final_train_loss': history['train_loss'][-1],
            'final_val_loss': history['val_loss'][-1],
            'total_epochs': len(history['train_loss']),
            'parameters': model.count_parameters()['total'],
            'branch_weights_final': history['branch_weights'][-1] if history['branch_weights'] else [0.5, 0.5]
        }
        
        with open(horizon_dir / 'v5_stacking_history.json', 'w') as f:
            json.dump(history_data, f, indent=2)
        
        # Save training log CSV
        log_df = pd.DataFrame({
            'epoch': range(1, len(history['train_loss']) + 1),
            'train_loss': history['train_loss'],
            'val_loss': history['val_loss'],
            'train_mae': history['train_mae'],
            'val_mae': history['val_mae'],
            'lr': history['lr']
        })
        log_df.to_csv(horizon_dir / f'v5_stacking_training_log_h{horizon}.csv', index=False)
        
        # Collect metrics for CSV
        for h_key, h_metrics in metrics.items():
            if h_key.startswith('H'):
                h_num = int(h_key[1:])
                row = {
                    'TotalHorizon': horizon,
                    'Experiment': 'BASIC_KCE',
                    'Model': 'V5_STACKING',
                    'H': h_num,
                    **h_metrics
                }
                all_metrics.append(row)
        
        # Store results
        experiment_state['results'][f'H{horizon}'] = {
            'metrics': metrics,
            'history': history_data
        }
        
        # Print summary
        print(f"\nH{horizon} Results Summary:")
        for h_key, h_metrics in metrics.items():
            if h_key.startswith('H'):
                print(f"  {h_key}: RMSE={h_metrics['RMSE']:.2f}mm, "
                      f"MAE={h_metrics['MAE']:.2f}mm, R²={h_metrics['R^2']:.4f}")
        print(f"  Branch weights: ConvLSTM={metrics['branch_weights']['w_convlstm']:.2%}, "
              f"GNN={metrics['branch_weights']['w_gnn']:.2%}")
        
        # Cleanup
        del model, trainer
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    # Save consolidated metrics CSV
    if all_metrics:
        metrics_df = pd.DataFrame(all_metrics)
        metrics_df.to_csv(output_root / 'metrics_spatial_v5_all_horizons.csv', index=False)
        print(f"\nConsolidated metrics saved to: {output_root / 'metrics_spatial_v5_all_horizons.csv'}")
    
    # Save experiment state
    with open(output_root / 'experiment_state_v5.json', 'w') as f:
        json.dump(experiment_state, f, indent=2, default=str)
    
    return experiment_state, metrics_df if all_metrics else None

print("Main training function defined")

In [ ]:
# =============================================================================
# SECTION 8.2: EXECUTE TRAINING
# =============================================================================

# Run experiments
print("\n" + "#"*70)
print("#" + " "*20 + "V5 GNN-ConvLSTM STACKING" + " "*20 + "#")
print("#"*70)

if CONFIG.light_mode:
    print("\n>>> LIGHT MODE ENABLED - Using reduced grid for testing <<<")
    print(f">>> Grid size: {CONFIG.light_grid_size}x{CONFIG.light_grid_size} <<<\n")

experiment_state, metrics_df = run_v5_experiments(
    ds=ds,
    config=CONFIG,
    edge_index=edge_index,
    edge_weight=edge_weight,
    output_root=OUTPUT_ROOT
)

print("\n" + "="*70)
print("V5 TRAINING COMPLETE")
print("="*70)

## 9. Results Export and Visualization

In [ ]:
# =============================================================================
# SECTION 9: RESULTS VISUALIZATION (700 DPI)
# =============================================================================

def plot_v5_results(metrics_df: pd.DataFrame, experiment_state: Dict, output_dir: Path):
    """Generate V5 results visualizations at 700 DPI."""
    
    if metrics_df is None or len(metrics_df) == 0:
        print("No metrics to plot")
        return
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    
    # 1. RMSE by Horizon
    ax1 = axes[0, 0]
    horizons = sorted(metrics_df['H'].unique())
    rmse_by_h = metrics_df.groupby('H')['RMSE'].mean()
    ax1.bar(rmse_by_h.index, rmse_by_h.values, color='steelblue', edgecolor='black')
    ax1.set_xlabel('Prediction Horizon (months)', fontsize=12)
    ax1.set_ylabel('RMSE (mm)', fontsize=12)
    ax1.set_title('V5 Stacking: RMSE by Forecast Horizon', fontsize=14, fontweight='bold')
    ax1.grid(axis='y', alpha=0.3)
    
    # Add target line
    ax1.axhline(y=85, color='red', linestyle='--', label='Target: 85mm')
    ax1.legend()
    
    # 2. R² by Horizon
    ax2 = axes[0, 1]
    r2_by_h = metrics_df.groupby('H')['R^2'].mean()
    ax2.bar(r2_by_h.index, r2_by_h.values, color='forestgreen', edgecolor='black')
    ax2.set_xlabel('Prediction Horizon (months)', fontsize=12)
    ax2.set_ylabel('R²', fontsize=12)
    ax2.set_title('V5 Stacking: R² by Forecast Horizon', fontsize=14, fontweight='bold')
    ax2.grid(axis='y', alpha=0.3)
    
    # Add target line
    ax2.axhline(y=0.65, color='red', linestyle='--', label='Target: 0.65')
    ax2.legend()
    
    # 3. Bias by Horizon
    ax3 = axes[1, 0]
    bias_by_h = metrics_df.groupby('H')['mean_bias_mm'].mean()
    colors = ['red' if b < 0 else 'blue' for b in bias_by_h.values]
    ax3.bar(bias_by_h.index, bias_by_h.values, color=colors, edgecolor='black')
    ax3.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax3.set_xlabel('Prediction Horizon (months)', fontsize=12)
    ax3.set_ylabel('Mean Bias (mm)', fontsize=12)
    ax3.set_title('V5 Stacking: Bias by Forecast Horizon', fontsize=14, fontweight='bold')
    ax3.grid(axis='y', alpha=0.3)
    
    # 4. Branch Weights Evolution
    ax4 = axes[1, 1]
    
    # Extract branch weights from experiment state
    horizon_labels = []
    w_convlstm = []
    w_gnn = []
    
    for h_key, results in experiment_state['results'].items():
        if 'metrics' in results and 'branch_weights' in results['metrics']:
            horizon_labels.append(h_key)
            w_convlstm.append(results['metrics']['branch_weights']['w_convlstm'])
            w_gnn.append(results['metrics']['branch_weights']['w_gnn'])
    
    x = np.arange(len(horizon_labels))
    width = 0.35
    
    ax4.bar(x - width/2, w_convlstm, width, label='ConvLSTM', color='coral', edgecolor='black')
    ax4.bar(x + width/2, w_gnn, width, label='GNN-TAT', color='teal', edgecolor='black')
    
    ax4.set_xlabel('Horizon', fontsize=12)
    ax4.set_ylabel('Branch Weight', fontsize=12)
    ax4.set_title('V5 Meta-Learner: Branch Weight Distribution', fontsize=14, fontweight='bold')
    ax4.set_xticks(x)
    ax4.set_xticklabels(horizon_labels)
    ax4.legend()
    ax4.grid(axis='y', alpha=0.3)
    ax4.set_ylim(0, 1)
    
    plt.tight_layout()
    
    # Save at 700 DPI
    fig_path = output_dir / 'v5_results_summary.png'
    fig.savefig(fig_path, dpi=700, bbox_inches='tight', facecolor='white')
    plt.show()
    
    print(f"\nVisualization saved to: {fig_path} (700 DPI)")

# Generate visualizations
plot_v5_results(metrics_df, experiment_state, OUTPUT_ROOT)

In [ ]:
# =============================================================================
# SECTION 9.2: TRAINING CURVES VISUALIZATION
# =============================================================================

def plot_training_curves(experiment_state: Dict, output_dir: Path):
    """Plot training curves for all horizons."""
    
    n_horizons = len(experiment_state['results'])
    if n_horizons == 0:
        print("No training results to plot")
        return
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.flatten()
    
    colors = ['steelblue', 'coral', 'forestgreen', 'purple']
    
    for idx, (h_key, results) in enumerate(experiment_state['results'].items()):
        if idx >= 4:
            break
            
        ax = axes[idx]
        
        # Load training log
        h_num = h_key.replace('H', '')
        log_path = output_dir / f'h{h_num}' / 'BASIC_KCE' / 'training_metrics' / f'v5_stacking_training_log_h{h_num}.csv'
        
        if log_path.exists():
            log_df = pd.read_csv(log_path)
            
            ax.plot(log_df['epoch'], log_df['train_loss'], label='Train Loss', color=colors[idx], alpha=0.8)
            ax.plot(log_df['epoch'], log_df['val_loss'], label='Val Loss', color=colors[idx], linestyle='--', alpha=0.8)
            
            # Mark best epoch
            best_epoch = results['history']['best_epoch'] + 1
            best_val = results['history']['best_val_loss']
            ax.scatter([best_epoch], [best_val], color='red', s=100, zorder=5, marker='*', label=f'Best: {best_val:.4f}')
        
        ax.set_xlabel('Epoch', fontsize=10)
        ax.set_ylabel('Loss', fontsize=10)
        ax.set_title(f'Training Curves - {h_key}', fontsize=12, fontweight='bold')
        ax.legend(fontsize=8)
        ax.grid(alpha=0.3)
    
    plt.tight_layout()
    
    # Save at 700 DPI
    fig_path = output_dir / 'v5_training_curves.png'
    fig.savefig(fig_path, dpi=700, bbox_inches='tight', facecolor='white')
    plt.show()
    
    print(f"Training curves saved to: {fig_path} (700 DPI)")

# Plot training curves
plot_training_curves(experiment_state, OUTPUT_ROOT)

## 10. Summary and Next Steps

In [ ]:
# =============================================================================
# SECTION 10: SUMMARY AND RESULTS
# =============================================================================

print("\n" + "="*70)
print("V5 GNN-ConvLSTM STACKING - EXPERIMENT SUMMARY")
print("="*70)

# Print configuration
print("\n[Configuration]")
print(f"  Light Mode: {CONFIG.light_mode}")
if CONFIG.light_mode:
    print(f"  Grid Size: {CONFIG.light_grid_size}x{CONFIG.light_grid_size}")
print(f"  GNN Type: {CONFIG.gnn_type}")
print(f"  Horizons: {CONFIG.enabled_horizons}")
print(f"  Epochs: {CONFIG.epochs}")
print(f"  Batch Size: {CONFIG.batch_size}")

# Print results summary
if metrics_df is not None and len(metrics_df) > 0:
    print("\n[Results Summary]")
    print("-"*70)
    print(metrics_df[['TotalHorizon', 'H', 'RMSE', 'MAE', 'R^2', 'mean_bias_mm']].to_string(index=False))
    print("-"*70)
    
    # Overall averages
    print("\n[Overall Averages]")
    print(f"  RMSE: {metrics_df['RMSE'].mean():.2f} mm (Target: < 85 mm)")
    print(f"  MAE: {metrics_df['MAE'].mean():.2f} mm")
    print(f"  R²: {metrics_df['R^2'].mean():.4f} (Target: > 0.65)")
    print(f"  Bias: {metrics_df['mean_bias_mm'].mean():.2f} mm")

# Print branch weights
print("\n[Branch Weights (Meta-Learner)]")
for h_key, results in experiment_state['results'].items():
    if 'metrics' in results and 'branch_weights' in results['metrics']:
        weights = results['metrics']['branch_weights']
        print(f"  {h_key}: ConvLSTM={weights['w_convlstm']:.1%}, GNN={weights['w_gnn']:.1%}")

# Output files
print("\n[Output Files]")
print(f"  Output Directory: {OUTPUT_ROOT}")
for path in OUTPUT_ROOT.glob('*'):
    if path.is_file():
        print(f"  - {path.name}")

# Next steps
print("\n[Next Steps]")
if CONFIG.light_mode:
    print("  1. Set light_mode=False for full grid validation (61x65)")
    print("  2. Run full experiments on Colab GPU")
print("  3. Compare results with V4 baseline (R²=0.628, RMSE=92.12mm)")
print("  4. Analyze branch contributions by elevation regime")
print("  5. Run ablation studies (A1-A6 from spec.md)")
print("  6. Integrate with Paper-5 documentation")

print("\n" + "="*70)
print("V5 NOTEBOOK EXECUTION COMPLETE")
print("="*70)

In [ ]:
# =============================================================================
# SECTION 10.2: CLEANUP
# =============================================================================

# Memory cleanup
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"GPU Memory freed. Current usage: {torch.cuda.memory_allocated() / 1e6:.1f} MB")

print("\nNotebook execution completed successfully.")